In [1]:
import pandas as pd
import numpy as np
import os
import csv
import json
from pathlib import Path
os.environ['cuda_visible_device'] = '0'

In [3]:
label_file = '../genomic_data/blca_msk_tcga_2020_cbioportal/data_log2_cna.txt'
label_pd = pd.read_csv(label_file, sep="\t")

In [22]:
# label_pd['Hugo_Symbol']
# label_pd['TCGA-DK-A2I6-01']
# To get all patient values
labels = label_pd.loc[:, ['Hugo_Symbol', 'TCGA-DK-A2I6-01']]
label_pd.loc[df['Hugo_Symbol']=='AGRN']

,Hugo_Symbol,TCGA-DK-A2I6-01
0,ACAP3,0.350
1,ACTRT2,0.350
2,AGRN,0.350
3,ANKRD65,0.350
4,ATAD3A,0.350
...,...,...
25123,hsa-mir-513c,0.197
25124,hsa-mir-513b,0.197
25125,hsa-mir-513a-1,0.197
25126,hsa-mir-513a-2,0.197


In [29]:
labels.loc[labels['Hugo_Symbol']=='AGRN']

,Hugo_Symbol,TCGA-DK-A2I6-01
2,AGRN,0.35


In [31]:
def get_data_RNA(df, gene_names, patient_ids):
    patient_dicts = []
    vals = []
    for col in df.columns:
        if ('TCGA' in col):
            if p in patient_ids and '-01A' in col:  # only unhealthy samples (healthy will have >=10 instead of 01)
                curr_patient = {}
                curr_patient['id'] = p
                for gene in gene_names:
                    v = float(df.loc[df['Hugo_Symbol']==gene, col].values[0])
                    curr_patient[gene] = v
                patient_dicts.append(curr_patient)
                vals.append(v)
    vals = np.asarray(vals)
    mini = np.min(vals)
    maxi = np.max(vals)
    for p in patient_dicts:
        p['label'] = (p['label'] - mini) / (maxi - mini)
    return patient_dicts

    

In [29]:
label_type = 'RNA'
gene_names = ['MKI67|4288']
data_path = Path('../data/')
patient_id_file = Path('../patient_id_all.txt')
patient_id = []
with open(patient_id_file, 'r') as f:
    patient_id = f.read().splitlines()
patient_id

['TCGA-A2-A25A',
 'TCGA-D8-A1XW',
 'TCGA-BH-A203',
 'TCGA-D8-A27M',
 'TCGA-AC-A2FO',
 'TCGA-E2-A107',
 'TCGA-AR-A0TP',
 'TCGA-E9-A1R6',
 'TCGA-A2-A04T',
 'TCGA-A2-A25D',
 'TCGA-BH-A18M',
 'TCGA-E2-A14P',
 'TCGA-C8-A27A',
 'TCGA-GM-A2DD',
 'TCGA-D8-A1JU',
 'TCGA-E9-A5FK',
 'TCGA-BH-A0HQ',
 'TCGA-D8-A1JL',
 'TCGA-A8-A08X']

In [4]:
if (label_type == 'RNA'):
    label_path = data_path/'EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv'
elif (label_type == 'MiRNA'):
    label_path = data_path/'pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.csv'

In [5]:
content = pd.read_csv(label_path, sep='\t', header=0)

In [8]:
content.head()

,gene_id,TCGA-OR-A5J1-01A-11R-A29S-07,TCGA-OR-A5J2-01A-11R-A29S-07,TCGA-OR-A5J3-01A-11R-A29S-07,TCGA-OR-A5J5-01A-11R-A29S-07,TCGA-OR-A5J6-01A-31R-A29S-07,TCGA-OR-A5J7-01A-11R-A29S-07,TCGA-OR-A5J8-01A-11R-A29S-07,TCGA-OR-A5J9-01A-11R-A29S-07,TCGA-OR-A5JA-01A-11R-A29S-07,...,TCGA-CG-4449-01A-01R-1157-13,TCGA-CG-4462-01A-01R-1157-13,TCGA-CG-4465-01A-01R-1157-13,TCGA-CG-4466-01A-01R-1157-13,TCGA-CG-4469-01A-01R-1157-13,TCGA-CG-4472-01A-01R-1157-13,TCGA-CG-4474-01A-02R-1157-13,TCGA-CG-4475-01A-01R-1157-13,TCGA-CG-4476-01A-01R-1157-13,TCGA-CG-4477-01A-01R-1157-13
0,?|100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,?|100133144,3.2661,2.6815,1.7301,0.0000,0.0000,1.1673,1.4422,0.0000,4.4556,...,4.358154,5.676995,5.219350,14.846708,20.115492,6.997533,18.311906,12.057112,18.628740,17.874417
2,?|100134869,3.9385,8.9948,6.5650,1.5492,4.4709,6.0529,2.2876,1.3599,5.0581,...,2.656360,3.342794,2.423442,5.055287,11.626054,13.654193,7.417109,11.585177,11.482418,14.919338
3,?|10357,149.1350,81.0777,86.4879,53.9117,66.9063,103.5060,94.9316,78.1955,69.2389,...,633.299781,294.018042,686.569179,563.573453,1039.307597,639.238135,742.479964,506.336449,712.452165,703.713324
4,?|10431,2034.1000,1304.9300,1054.6600,2350.8900,1257.9900,1866.4300,995.0270,1762.1200,1213.5300,...,1202.538277,644.002317,1181.884532,663.885074,647.530395,1297.152549,1152.909807,1375.495774,971.893874,1736.988111


In [32]:
v = content.loc[content['gene_id']==gene_names[0]].values
v

array([['MKI67|4288', 108.069, 964.702, ..., 3967.76273146212,
        6036.78925159929, 9573.92802685253]], dtype=object)

In [33]:
values = get_data_RNA(content, gene_names, patient_id)

In [34]:
values


[{'id': 'TCGA-A2-A04T', 'label': 0.7430635852434544},
 {'id': 'TCGA-A2-A25A', 'label': 0.15077798506891874},
 {'id': 'TCGA-A2-A25D', 'label': 0.12364211816034353},
 {'id': 'TCGA-A8-A08X', 'label': 0.48219917726310885},
 {'id': 'TCGA-AC-A2FO', 'label': 0.03798256479364855},
 {'id': 'TCGA-AR-A0TP', 'label': 0.6563511931897211},
 {'id': 'TCGA-BH-A0HQ', 'label': 0.10145684067433453},
 {'id': 'TCGA-BH-A18M', 'label': 0.182844436641281},
 {'id': 'TCGA-BH-A203', 'label': 0.268023522772123},
 {'id': 'TCGA-C8-A27A', 'label': 1.0},
 {'id': 'TCGA-D8-A1JL', 'label': 0.3708903459152283},
 {'id': 'TCGA-D8-A1JU', 'label': 0.0},
 {'id': 'TCGA-D8-A1XW', 'label': 0.14634869612512527},
 {'id': 'TCGA-D8-A27M', 'label': 0.542950099423651},
 {'id': 'TCGA-E2-A107', 'label': 0.2839555480138922},
 {'id': 'TCGA-E2-A14P', 'label': 0.4134804773559074},
 {'id': 'TCGA-E9-A1R6', 'label': 0.15244897080223083},
 {'id': 'TCGA-E9-A5FK', 'label': 0.02614115969704793},
 {'id': 'TCGA-GM-A2DD', 'label': 0.14229126034451958}

In [35]:
# header = ['id', 'label']
# fname = f'../data/labels/label_{gene_name}.csv'
# with open(fname, 'w', newline='') as myfile:
#     writer = csv.DictWriter(myfile, fieldnames=header)
#     writer.writeheader()
#     writer.writerows(values)

with open('../genomic_data/label_summary.json', 'w') as fwrite:
    json.dump(fwrite, values, indent=4)

In [ ]:
# some globals, can be changed to customize
target_mag = 20
threshold = 0.75
patch_size = 512
num_tiles_per_person = 200
data_dir = Path('../slide_data/')
save_dir_root = Path('../tile_data/')
if save_dir_root.exists():
    shutil.rmtree(save_dir_root)
# save_dir_root.mkdir()
all_slides, all_patients = get_all_patient_slides(data_dir)
all_slides


['../data/slides/TCGA-BH-A0HQ/TCGA-BH-A0HQ-01Z-00-DX1.0921FCEF-20A2-4D4B-A198-91AF9F6C814C.svs',
 '../data/slides/TCGA-E2-A14P/TCGA-E2-A14P-01Z-00-DX1.663B02FF-C64B-41A6-8685-FD61CD76F9C6.svs',
 '../data/slides/TCGA-D8-A27M/TCGA-D8-A27M-01Z-00-DX1.3020D223-2400-4A2D-8BFE-08A5B78FE13B.svs',
 '../data/slides/TCGA-E9-A5FK/TCGA-E9-A5FK-01Z-00-DX1.3F002C99-B4CA-4068-A816-EAC370A2E8C4.svs',
 '../data/slides/TCGA-AR-A0TP/TCGA-AR-A0TP-01Z-00-DX1.58E6A3A6-D1D7-4AE5-87A5-E823F04CBB2F.svs',
 '../data/slides/TCGA-D8-A1JU/TCGA-D8-A1JU-01Z-00-DX1.355D93B4-E69E-417C-B3D1-3E1AAF1E02FE.svs',
 '../data/slides/TCGA-D8-A1JL/TCGA-D8-A1JL-01Z-00-DX1.FE3F0C6B-F98A-4036-BF9A-25A8CC66B1FD.svs',
 '../data/slides/TCGA-BH-A18M/TCGA-BH-A18M-01Z-00-DX1.56E6935A-62CB-4CBE-BDCA-60DA56422CE0.svs',
 '../data/slides/TCGA-C8-A27A/TCGA-C8-A27A-01Z-00-DX1.0E26C46D-CD65-40F3-8976-EB4415582934.svs',
 '../data/slides/TCGA-GM-A2DD/TCGA-GM-A2DD-01Z-00-DX1.01332FF3-7488-41B7-A515-7841715DE92B.svs',
 '../data/slides/TCGA-A2-A04T/